In [103]:
import glob
import pandas as pd
import numpy as np
df = None
for fname in glob.glob('results/results*.txt'):
    newdf = pd.read_csv(fname,header=None,index_col=False,names=['epsilon','steps','-','no dp','-','integral','-','cloaking'])
    if df is None:
        df = newdf
    else:
        df = pd.concat([df,newdf])
        #df.append(newdf, ignore_index=True)
df.drop('-', axis=1, inplace=True)

summary = {}
nodp = df['no dp']
summary['nodp'] = {'mean':np.mean(nodp), 'std':np.std(nodp), 'n':len(nodp)}
summary['cloaking'] = {}
summary['integral'] = {}

for eps in df['epsilon'].unique():
    cloaking = df[df['epsilon']==eps]['cloaking']
    summary['cloaking'][eps] = {'mean':np.mean(cloaking), 'std':np.std(cloaking), 'n':len(cloaking)}
    summary['integral'][eps] = {}
    for step in df['steps'].unique():
        integral = df[(df['epsilon']==eps) & (df['steps']==step)]['integral']
        summary['integral'][eps][step] = {'mean':np.mean(integral), 'std':np.std(integral), 'n':len(integral)}

In [106]:
def add_ste(s):
    s['ste'] = s['std']/np.sqrt(s['n'])
    
for k in summary['cloaking']:
    add_ste(summary['cloaking'][k])
        
for k in summary['integral']:
    for j in summary['integral'][k]:
        add_ste(summary['integral'][k][j])

add_ste(summary['nodp'])
        


array([ 1,  3,  6, 10])

In [142]:
prepend = "\\begin{tabular}{ l l l l l l }\n"
prepend += "\\hline\n"
prepend += "Epsilon & Cloaking & \\multicolumn{4}{ c }{Binning with Integral Kernel} \\\\\n"
prepend += " &  "
for step in np.sort(df['steps'].unique()):
    if step==1:
        add_s = ""
    else:
        add_s = "s"
    prepend += "& %d bin%s " % (step,add_s)
prepend += "\\\\ \n"
prepend += "\\hline\n"
append = "\\hline\n"
append += "\\end{tabular}\n"
rows = []
for eps in np.sort(df['epsilon'].unique())[-1::-1]:
    
    row = ""
    row += "$%0.2f$ & $%0.2f \\pm %0.2f$ " % (eps, summary['cloaking'][eps]['mean'], summary['cloaking'][eps]['ste'])
    for step in np.sort(df['steps'].unique()):
        row += "& $%0.2f \\pm %0.2f$ " % (summary['integral'][eps][step]['mean'], summary['integral'][eps][step]['ste'])
    row += " \\\\"
    rows.append(row)

output = prepend + "\n".join(rows) + "\n" + append
 
print output    

\begin{tabular}{ l l l l l l }
\hline
Epsilon & Cloaking & \multicolumn{4}{ c }{Binning with Integral Kernel} \\
 &  & 1 bin & 3 bins & 6 bins & 10 bins \\ 
\hline
$1.00$ & $0.89 \pm 0.04$ & $1.00 \pm 0.02$ & $1.30 \pm 0.06$ & $1.30 \pm 0.06$ & $1.17 \pm 0.02$  \\
$0.50$ & $1.21 \pm 0.05$ & $1.01 \pm 0.01$ & $1.35 \pm 0.12$ & $1.44 \pm 0.13$ & $1.34 \pm 0.07$  \\
$0.20$ & $2.40 \pm 0.12$ & $1.02 \pm 0.01$ & $2.74 \pm 0.41$ & $2.21 \pm 0.20$ & $1.64 \pm 0.21$  \\
$0.10$ & $4.82 \pm 0.28$ & $0.99 \pm 0.02$ & $4.59 \pm 0.69$ & $3.91 \pm 0.63$ & $2.68 \pm 0.31$  \\
$0.01$ & $48.44 \pm 3.22$ & $1.05 \pm 0.02$ & $13.65 \pm 2.50$ & $28.13 \pm 6.29$ & $22.72 \pm 2.92$  \\
\hline
\end{tabular}



In [143]:
summary['nodp']

{'mean': 0.66955978125,
 'n': 320,
 'std': 0.0859667785121942,
 'ste': 0.0048056890139983616}